# What is IDOM?

IDOM connects your Python web framework of choice to a ReactJS frontend, allowing you to create **interactive websites without needing JavaScript!**

Following ReactJS styling, web elements are combined into [reusable "components"](https://idom-docs.herokuapp.com/docs/guides/creating-interfaces/your-first-components/index.html#parametrizing-components). These components can utilize [hooks](https://idom-docs.herokuapp.com/docs/reference/hooks-api.html) and [events](https://idom-docs.herokuapp.com/docs/guides/adding-interactivity/responding-to-events/index.html#async-event-handlers) to create infinitely complex web pages.

When needed, IDOM can [use components directly from NPM](https://idom-docs.herokuapp.com/docs/guides/escape-hatches/javascript-components.html#dynamically-loaded-components). For additional flexibility, components can also be [fully developed in JavaScript](https://idom-docs.herokuapp.com/docs/guides/escape-hatches/javascript-components.html#custom-javascript-components).


# Getting Started

The first thing we need must do in order to run IDOM in a Jupyter Notebook is `import idom_jupyter`. After that, you'll be able to use IDOM components like any other [Jupyter Widget](https://ipywidgets.readthedocs.io/en/latest/#).

In [2]:
import idom_jupyter

# At a Glance

To get a rough idea of how to write apps in IDOM, take a look at the tiny [“hello world”](https://en.wikipedia.org/wiki/%22Hello,_World!%22_program) application below:

In [3]:
from idom import component, html, run


@component
def App():
    return html.h1("Hello, World!")


App()

App(7fb016a9afe0)

LayoutWidget(Layout(App(7fb016a9afe0)))

# Creating Interfaces

IDOM is a Python package for making user interfaces (UI). These interfaces are built from small elements of functionality like buttons text and images. IDOM allows you to combine these elements into reusable “components”. Once you learn how these UI elements are created and organized into components you'll be able to do things like create interfaces from raw data:

<a href="https://idom-docs.herokuapp.com/docs/creating-interfaces/index.html" target="_blank"><button>Read More!</button></a>

In [ ]:
from idom import component, html


@component
def DataList(items, filter_by_priority=None, sort_by_priority=False):
    if filter_by_priority is not None:
        items = [i for i in items if i["priority"] <= filter_by_priority]
    if sort_by_priority:
        items = list(sorted(items, key=lambda i: i["priority"]))
    list_item_elements = [html.li(i["text"], key=i["id"]) for i in items]
    return html.ul(list_item_elements)


@component
def TodoList():
    tasks = [
        {"id": 0, "text": "Make breakfast", "priority": 0},
        {"id": 1, "text": "Feed the dog", "priority": 0},
        {"id": 2, "text": "Do laundry", "priority": 2},
        {"id": 3, "text": "Go on a run", "priority": 1},
        {"id": 4, "text": "Clean the house", "priority": 2},
        {"id": 5, "text": "Go to the grocery store", "priority": 2},
        {"id": 6, "text": "Do some coding", "priority": 1},
        {"id": 7, "text": "Read a book", "priority": 1},
    ]
    return html.section(
        html.h1("My Todo List"),
        DataList(tasks, filter_by_priority=1, sort_by_priority=True),
    )


TodoList()

# Adding Interactivity

Components often need to change what’s on the screen as a result of an interaction. For example, typing into the form should update the input field, clicking a “Comment” button should bring up a text input field, clicking “Buy” should put a product in the shopping cart. Components need to “remember” things like the current input value, the current image, the shopping cart. In IDOM, this kind of component-specific memory is created and updated with a “hook” called use_state() that creates a state variable and state setter respectively:

<a href="https://idom-docs.herokuapp.com/docs/adding-interactivity/index.html" target="_blank"><button>Read More!</button></a>

In [8]:
import json
from pathlib import Path

from idom import component, use_state, html


DATA_PATH = Path().parent / "data" / "gallery-data.json"
sculpture_data = json.loads(DATA_PATH.read_text())


@component
def Gallery():
    index, set_index = use_state(0)

    def handle_click(event):
        set_index(index + 1)

    bounded_index = index % len(sculpture_data)
    sculpture = sculpture_data[bounded_index]
    alt = sculpture["alt"]
    artist = sculpture["artist"]
    description = sculpture["description"]
    name = sculpture["name"]
    url = sculpture["url"]

    return html.div(
        html.button({"onClick": handle_click}, "Next"),
        html.h2(name, " by ", artist),
        html.p(f"({bounded_index + 1} or {len(sculpture_data)})"),
        html.img({"src": url, "alt": alt, "style": {"height": "200px"}}),
        html.p(description),
    )


Gallery()

Gallery(7fb017f937c0)

LayoutWidget(Layout(Gallery(7fb017f937c0)))

# Using IDOM With Jupyter Widgets

While you can use IDOM components independently, it may also be useful to integrate them with the rest of the Jupyter Widget ecosystem. Let's consider an IDOM component that responds to and displays changes from an `ipywidgets.IntSlider`. The IDOM component will need to accept an `IntSlider` instance as one of its arguments, declare state that will track the slider's value, and register a lister that will update that state via the slider's `IntSlider.observe()` method using an ["effect"](https://idom-docs.herokuapp.com/docs/reference/hooks-api.html#use-effect):

In [14]:
from idom import use_effect


@component
def SliderObserver(slider):
    value, set_value = use_state(0)

    @use_effect
    def register_observer():

        def handle_change(change):
            set_value(change["new"])

        # observe the slider's value
        slider.observe(handle_change, "value")
        # unobserve the slider's value if this component is no longer displayed
        return lambda: slider.unobserve(handle_change, "value")

    return html.p(f"IDOM observes the value to be: ", value)

Now you'll need to display the `SliderObserver` component as well as an `IntSlider` widget. To do this, you'll want wrap the component in a `idom_jupyter.LayoutWidget` instance before using it alongside other Jupyter Widgets. Specifically, you'll be displaying the `SliderObserver` and `IntSlider` in a `Box`:


In [15]:
from ipywidgets import Box, IntSlider
from idom_jupyter import LayoutWidget

slider = IntSlider(readout=False)
slider_observer = LayoutWidget(SliderObserver(slider))

Box([slider, slider_observer])

Box(children=(IntSlider(value=0, readout=False), LayoutWidget(Layout(SliderObserver(7fb01611ed40, slider=IntSl…

If it becomes painful to wrap every IDOM component in a `LayoutWidget` you can create an alternate `LayoutWidget` constructor using `idom_jupyter.widgetize`:

In [16]:
from idom_jupyter import widgetize

SliderObserverWidget = widgetize(SliderObserver)

slider = IntSlider(readout=False)
slider_observer = SliderObserverWidget(slider)

Box([slider, slider_observer])

Box(children=(IntSlider(value=0, readout=False), LayoutWidget(Layout(SliderObserver(7fb01611fa00, slider=IntSl…

It's worth noting that, while IDOM can be used with Jupyter Widgets, the reverse is not true. That is, **you cannot using a Jupyter Widget inside an IDOM component**. If this is a capability you would find useful, please [start a discussion](https://github.com/idom-team/idom/discussions). The IDOM team would be very interested to hear how allowing Jupyter Widgets to be used with IDOM could facilitate your work.

# Javascript Integration

While IDOM is a great tool for displaying HTML and responding to browser events with pure Python, there are other projects which already allow you to do this inside Jupyter Notebooks or in standard web apps. The real power of IDOM comes from its ability to seamlessly leverage the existing Javascript ecosystem:

<a href="https://idom-docs.herokuapp.com/docs/escape-hatches/javascript-components.html" target="_blank"><button>Read More!</button></a>

In [ ]:
from idom import component, web


victory = web.module_from_template("react", "victory-bar", fallback="⌛")
VictoryBar = web.export(victory, "VictoryBar")


@component
def Demo():
    bar_style = {"parent": {"width": "500px"}, "data": {"fill": "royalblue"}}
    return VictoryBar({"style": bar_style})


Demo()

# [Learn More!](https://idom-docs.herokuapp.com)